In [1]:
# This is an attempt to solve the screening task to find numbers that are less than a reference value using Quantum programming
# This task is a part of skills assessment for the QOSF's Quantum Computing Mentorship Program - cohort 9
# Programming this in the Quantum simulator provided by IBMQ the open source platform 

In [2]:
# Import the relevant Libraries from qiskit package
#%pip install qiskit
from qiskit.circuit import QuantumCircuit
from qiskit.circuit.library import IntegerComparator
from qiskit import transpile
from qiskit_aer import Aer

In [3]:
# This is the main calling function that takes in an integer value 'k' that is the positive number to compare in list of numnbers list_n
# This function calls other anciliary functions to build and run the quantum circuit and retrieve the numbers identified as lesser than k
# The length of the binary representation for the largest number in the list (list_n) determines the number of qubits required in the comparator circuit and hence the depth of the circuit
# For E.g., if the largest number is in the range between 8 and 15, then, length of binary string is 4 and hence 4 qubits will be needed, else, if
# largest number is between 16 and 31, then the length of binary string is 5 and hence 5 qubits would be required
def less_than_k(k,list_n):
    maxnum=max(list_n)
    size = len(bin(maxnum)[2:])
    circ = build_comparator_circuit(size, k)
    resultset=run_comparator_circuit(circ)
    return retrieve_num_less_than_k(resultset,list_n)

In [4]:
# Function to build and print the circuit, input arguments are size of the Quantum register needed and the value to be compared against
# This uses the IntegerComparator function which compares an integer against a static value - the flag 'geq' deteremines if the MSB needs to be set to 1 
# when number is greater or lesser (if geq is True, the MSB is set to 1 when number is greater than the value compared against and if false, the MSB is set to 1 when number is lesser thena the value compared against
def build_comparator_circuit(size, value):
    comp = IntegerComparator(size, value, geq=False, name='Comparator')
    circ = QuantumCircuit(2 * size, size + 1)
    circ.h(range(size))
    circ.append(comp, range(2 * size))
    circ.measure(range(size + 1), range(size + 1))
    print("Circuit ")
    print(circ)
    return circ

In [5]:
# Function to run the quantum circuit - this returns as result set containing the measurements made on the classical register
def run_comparator_circuit(ckt):
    backend=Aer.get_backend('aer_simulator')
    new_circuit = transpile(ckt, backend)
    job = backend.run(new_circuit)
    return job.result().get_counts()

In [6]:
# Function to retrieve the numbers less than the comparator and identify such numbers that exist in the list of numbers provided
def retrieve_num_less_than_k(resultset,listofnum):
    # Array to store the identified numbers for display
    listofnumlessthan_k=[]
    listofdecnums=[]

    for bitstring in sorted(resultset): 
        #Since the comparator property geq (greater than equal to) is set to false, the most significant bit is set to 1 where number is less than comparator number
        if bitstring[0] == "1":
            #retrieve the binary string for the number by removing the most significant bit 
            binstr=bitstring[1:]
            #convert to decimal representation
            #decnum = int(binstr,2)
            listofdecnums.append(int(binstr,2))
            #compare if the number identified exists in the original list and add it to the resultsetofnumlessthan_k
    [listofnumlessthan_k.append(i) for i in listofdecnums if i in listofnum]
    return listofnumlessthan_k

In [7]:
# Test Data sets
A= less_than_k(7,[4,9,11,14,1,13,6,15])
#A = less_than_k(25,[2,3,7,9,4,22,25,1,32])
#A = less_than_k(20,[1])
#A = less_than_k(22,[4,8,20,23,6,7])
#A = less_than_k(25,[8,100,22,50,67,5])

print("Result \n")
resstring = ','.join(str(x) for x in A)
print(resstring)


Circuit 
     ┌───┐┌─────────────┐┌─┐            
q_0: ┤ H ├┤0            ├┤M├────────────
     ├───┤│             │└╥┘┌─┐         
q_1: ┤ H ├┤1            ├─╫─┤M├─────────
     ├───┤│             │ ║ └╥┘┌─┐      
q_2: ┤ H ├┤2            ├─╫──╫─┤M├──────
     ├───┤│             │ ║  ║ └╥┘┌─┐   
q_3: ┤ H ├┤3            ├─╫──╫──╫─┤M├───
     └───┘│  Comparator │ ║  ║  ║ └╥┘┌─┐
q_4: ─────┤4            ├─╫──╫──╫──╫─┤M├
          │             │ ║  ║  ║  ║ └╥┘
q_5: ─────┤5            ├─╫──╫──╫──╫──╫─
          │             │ ║  ║  ║  ║  ║ 
q_6: ─────┤6            ├─╫──╫──╫──╫──╫─
          │             │ ║  ║  ║  ║  ║ 
q_7: ─────┤7            ├─╫──╫──╫──╫──╫─
          └─────────────┘ ║  ║  ║  ║  ║ 
c: 5/═════════════════════╩══╩══╩══╩══╩═
                          0  1  2  3  4 
Result 

1,4,6
